# Exercise 4

Work on this before the next lecture on 1 May. We will talk about questions, comments, and solutions during the exercise after the third lecture.

Please do form study groups! When you do, make sure you can explain everything in your own words, do not simply copy&paste from others.

The solutions to a lot of these problems can probably be found with Google. Please don't. You will not learn a lot by copy&pasting from the internet.

If you want to get credit/examination on this course please upload your work to **your GitHub repository** for this course **before** the next lecture starts and post a link to your repository [in this thread](). If you worked on things together with others please add their names to the notebook so we can see who formed groups.

---

These are some useful default imports for plotting and [`numpy`](http://www.numpy.org/)

In [1]:
%config InlineBackend.figure_format='retina'
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14
from sklearn.utils import check_random_state

## Pitfalls of estimating model performance

This question sets up a classification problem to illustrate a common pitfall in
evaluating model performance. To keep things simple the `y`s in this class room problem
are picked at random: there is no way for a classifier to learn how to model `y` based
on the features provided. This means we know what the true accuracy is: 0.5.

In [2]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, f_regression

np.random.seed(6450345)

A common task when building a new model is to select only those variables that are "best"
for the problem. This selection procedure can take many different shapes, here we will
compute the correlation of each feature with the target, select the 20 features that
have the highest correlation and use those in our gradient boosted tree ensemble.

We will then use cross validation to evaluate the performance.

In [3]:
def make_data(N=1000, n_vars=10,
              n_classes=2):
    X = np.random.normal(size=(N,n_vars))
    y = np.random.choice(n_classes, N)
    
    return X, y

X,y = make_data(N=2000, n_vars=50000)

select = SelectKBest(f_regression, k=20)
X_sel = select.fit_transform(X, y)

clf = GradientBoostingClassifier()
scores = cross_val_score(clf, X_sel, y, cv=5, n_jobs=8)

print("Scores on each subset:")
print(scores)
avg = (100*np.mean(scores), 100*np.std(scores)/np.sqrt(scores.shape[0]))
print("Average score and uncertainty: (%.2f +- %.3f)%%" % avg)

Scores on each subset:
[ 0.60349127  0.63092269  0.5975      0.63157895  0.65413534]
Average score and uncertainty: (62.35 +- 0.924)%


What just happened? We have a classifier that achieves and accuracy of ~60% but we know that the features are uncorrelated with the target. How did this happen? What mistake did we make?

What do we need to repair this? How do we know it is repaired? When the predicted performance
is close to what we know to be the true performance.

In [4]:
# your answer here

# Solution

The mistake is taking the cross-validation scores as an indication of the ability of the model to perform on new data. We need to instead split the data beforehand, keeping a small amount as testing data for the very end.

In [8]:
from sklearn.model_selection import train_test_split

X,y = make_data(N=2000, n_vars=50000)

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

select = SelectKBest(f_regression, k=20)
X_sel = select.fit_transform(X_train, y_train)

clf = GradientBoostingClassifier()
scores = cross_val_score(clf, X_sel, y_train, cv=5, n_jobs=8)

print("Wrong idea")
print("Scores on each subset:")
print(scores)
avg = (100*np.mean(scores), 100*np.std(scores)/np.sqrt(scores.shape[0]))
print("Average score and uncertainty: (%.2f +- %.3f)%%" % avg)


Wrong idea
Scores on each subset:
[ 0.61993769  0.571875    0.59375     0.578125    0.61442006]
Average score and uncertainty: (59.56 +- 0.853)%


In [12]:
print("Instead, the actual score is:")
X_sel_test = select.transform(X_test)
scores = cross_val_score(clf, X_sel_test, y_test, cv=15, n_jobs=8)
print(scores)
avg = (100*np.mean(scores), 100*np.std(scores)/np.sqrt(scores.shape[0]))
print("Average score and uncertainty: (%.2f +- %.3f)%%" % avg)

Instead, the actual score is:
[ 0.55555556  0.51851852  0.51851852  0.48148148  0.62962963  0.51851852
  0.51851852  0.44444444  0.37037037  0.59259259  0.55555556  0.51851852
  0.38461538  0.68        0.4       ]
Average score and uncertainty: (51.25 +- 2.180)%
